In [2]:
!pip install dmba
from dmba import classificationSummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 1.9 MB/s 
no display found. Using non-interactive Agg backend


In [3]:
import pandas as pd
import numpy as np
import io
from sklearn.linear_model import LogisticRegression # for running logistic regression model
from sklearn.model_selection import train_test_split # for splitting the data into train and test 
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf # we will use a function "logit" from "smf" to fit an logistic regression model

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import files
uploaded = files.upload()

Saving neo.csv to neo.csv


In [5]:
df = pd.read_csv(io.BytesIO(uploaded['neo.csv']))
df.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90836 entries, 0 to 90835
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  90836 non-null  int64  
 1   name                90836 non-null  object 
 2   est_diameter_min    90836 non-null  float64
 3   est_diameter_max    90836 non-null  float64
 4   relative_velocity   90836 non-null  float64
 5   miss_distance       90836 non-null  float64
 6   orbiting_body       90836 non-null  object 
 7   sentry_object       90836 non-null  bool   
 8   absolute_magnitude  90836 non-null  float64
 9   hazardous           90836 non-null  bool   
dtypes: bool(2), float64(5), int64(1), object(2)
memory usage: 5.7+ MB


In [7]:
df.columns.to_list()

['id',
 'name',
 'est_diameter_min',
 'est_diameter_max',
 'relative_velocity',
 'miss_distance',
 'orbiting_body',
 'sentry_object',
 'absolute_magnitude',
 'hazardous']

The column 'hazardous' is our outcome variable. We won't be using columns 'id', 'name', 'orbiting_body', and 'sentry_object' for the regression.

In [8]:
df['sentry_object'].value_counts()

False    90836
Name: sentry_object, dtype: int64

In [9]:
df['orbiting_body'].value_counts()

Earth    90836
Name: orbiting_body, dtype: int64

In [10]:
nasa_df = df.drop(columns=['id', 'name', 'orbiting_body', 'sentry_object'])
nasa_df.head()

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,1.198271,2.679415,13569.249224,5.483974e+07,16.73,False
1,0.265800,0.594347,73588.726663,6.143813e+07,20.00,True
2,0.722030,1.614507,114258.692129,4.979872e+07,17.83,False
3,0.096506,0.215794,24764.303138,2.543497e+07,22.20,False
4,0.255009,0.570217,42737.733765,4.627557e+07,20.09,True


In [11]:
nasa_df['hazardous'] = nasa_df['hazardous'].astype('int')
nasa_df['hazardous'].head()

0    0
1    1
2    0
3    0
4    1
Name: hazardous, dtype: int64

In [12]:
nasa_df.shape

(90836, 6)

In [13]:
nasa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90836 entries, 0 to 90835
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   est_diameter_min    90836 non-null  float64
 1   est_diameter_max    90836 non-null  float64
 2   relative_velocity   90836 non-null  float64
 3   miss_distance       90836 non-null  float64
 4   absolute_magnitude  90836 non-null  float64
 5   hazardous           90836 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 4.2 MB


For fitting an Explanatory Logistic Regression Formula we use the following formula:

exp_model = smf.logit(formula='dep_var ~ indep_var1 + indep_var2 + C(indep_var3)', data=df).fit()

In [14]:
log_reg = smf.logit(formula = "hazardous ~ est_diameter_min + est_diameter_max + relative_velocity + miss_distance + absolute_magnitude", data = nasa_df).fit_regularized()

/usr/local/lib/python3.7/dist-packages/statsmodels/discrete/discrete_model.py:1736: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.7/dist-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Iteration limit exceeded    (Exit mode 9)
            Current function value: inf
            Iterations: 1001
            Function evaluations: 11001
            Gradient evaluations: 1001


/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [15]:
print(log_reg.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/discrete/discrete_model.py:1736: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.7/dist-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


                           Logit Regression Results                           
Dep. Variable:              hazardous   No. Observations:                90836
Model:                          Logit   Df Residuals:                    90830
Method:                           MLE   Df Model:                            5
Date:                Wed, 06 Jul 2022   Pseudo R-squ.:                    -inf
Time:                        19:56:35   Log-Likelihood:                   -inf
converged:                      False   LL-Null:                       -28990.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -5.314e-05   1105.327  -4.81e-08      1.000   -2166.401    2166.401
est_diameter_min    2.749e-05        nan        nan        nan         nan         nan
est_diameter_max    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [16]:
logit_reg = smf.logit(formula = "hazardous ~ relative_velocity + miss_distance + absolute_magnitude", data = nasa_df).fit()

Optimization terminated successfully.
         Current function value: 0.242994
         Iterations 8


In [17]:
print(logit_reg.summary())

                           Logit Regression Results                           
Dep. Variable:              hazardous   No. Observations:                90836
Model:                          Logit   Df Residuals:                    90832
Method:                           MLE   Df Model:                            3
Date:                Wed, 06 Jul 2022   Pseudo R-squ.:                  0.2386
Time:                        19:56:36   Log-Likelihood:                -22073.
converged:                       True   LL-Null:                       -28990.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              9.5490      0.143     66.730      0.000       9.268       9.829
relative_velocity   8.037e-06   4.71e-07     17.064      0.000    7.11e-06    8.96e-06
miss_distance      -

## Predictive Model

In [18]:
X = nasa_df.drop(columns = ['hazardous'])
y = nasa_df['hazardous']
X.head()

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude
0,1.198271,2.679415,13569.249224,5.483974e+07,16.73
1,0.265800,0.594347,73588.726663,6.143813e+07,20.00
2,0.722030,1.614507,114258.692129,4.979872e+07,17.83
3,0.096506,0.215794,24764.303138,2.543497e+07,22.20
4,0.255009,0.570217,42737.733765,4.627557e+07,20.09


In [19]:
y.head()

0    0
1    1
2    0
3    0
4    1
Name: hazardous, dtype: int64

In [20]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 1)
train_X.shape

(72668, 5)

In [21]:
pred_model = LogisticRegression() #initialize the model
pred_model = pred_model.fit(train_X, train_y)

## Making Predictions

In [22]:
train_pred = pred_model.predict(train_X)
test_pred = pred_model.predict(test_X)

In [23]:
classificationSummary(train_y, train_pred)
classificationSummary(test_y, test_pred)

Confusion Matrix (Accuracy 0.9025)

       Prediction
Actual     0     1
     0 65586     0
     1  7082     0
Confusion Matrix (Accuracy 0.9032)

       Prediction
Actual     0     1
     0 16410     0
     1  1758     0
